In [ ]:
import pandas as pd

# Load file Excel
file_path = "/content/coalm 07.xlsx"
df = pd.read_excel(file_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/coalm 07.xlsx'

In [ ]:
import pandas as pd

# Path file Excel
file_path = "/content/coalm 07.xlsx"

# Membaca file Excel
df = pd.read_excel(file_path)

# Menampilkan 5 baris pertama
print("DataFrame yang dimuat:")
print(df.head())

In [ ]:
import pandas as pd

# Load Excel
file_path = "/content/coalm 07.xlsx"
df = pd.read_excel(file_path)

# Normalisasi nama kolom
df.columns = [col.lower() for col in df.columns]

# Simpan jumlah awal
before_cleaning = len(df)

# Daftar nilai yang harus dihapus (lowercase dan tanpa spasi ekstra)
to_exclude = ['areas, nes', 'north america and central america, nes', 'xxx, nes', 'bunkers', 'world', 'special categories', 'free zones']
to_exclude = [x.strip().lower() for x in to_exclude]

# Buat kolom bantu lowercase dan strip
df['reporterdesc_clean'] = df['reporterdesc'].str.strip().str.lower()
df['partnerdesc_clean'] = df['partnerdesc'].str.strip().str.lower()

# Filter 1: buang jika nilai termasuk ke dalam daftar to_exclude
filtered_df = df[
    ~df['reporterdesc_clean'].isin(to_exclude) &
    ~df['partnerdesc_clean'].isin(to_exclude)
]

# Filter 2: buang juga jika mengandung pola seperti (xxx, nes)
pattern = r"\(.*,\s*nes\)"
filtered_df = filtered_df[
    ~filtered_df['reporterdesc_clean'].str.contains(pattern, regex=True) &
    ~filtered_df['partnerdesc_clean'].str.contains(pattern, regex=True)
]

# Drop kolom bantu
filtered_df.drop(columns=['reporterdesc_clean', 'partnerdesc_clean'], inplace=True)

# Simpan jumlah akhir
after_cleaning = len(filtered_df)

# Info
print(f"Jumlah baris sebelum dibersihkan: {before_cleaning}")
print(f"Jumlah baris setelah dibersihkan: {after_cleaning}")
print(f"Jumlah baris yang dihapus: {before_cleaning - after_cleaning}")

# Simpan hasil akhir
filtered_df.to_excel("/content/cleaned_coalm 07.xlsx", index=False)

In [ ]:
import pandas as pd
import networkx as nx

# Load cleaned file
file_path = "/content/cleaned_coalm 07.xlsx"
df = pd.read_excel(file_path)

# Normalisasi kolom
df.columns = [col.lower() for col in df.columns]

# Pastikan kolom period berbentuk string
df['period'] = df['period'].astype(str)

# Nama kolom penting
period_col = 'period'
source_col = 'reporterdesc'
target_col = 'partnerdesc'
weight_col = 'fobvalue'  # pakai fobvalue sebagai bobot

# Simpan hasil analisis
results = []

for period in sorted(df[period_col].unique()):
    df_month = df[df[period_col] == period]

    # Buat directed graph berbobot
    G = nx.DiGraph()
    edges = list(zip(df_month[source_col], df_month[target_col], df_month[weight_col]))
    G.add_weighted_edges_from(edges)

    # Hitung metrik
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    avg_degree = sum(dict(G.degree()).values()) / num_nodes if num_nodes > 0 else 0
    avg_weighted_degree = sum(dict(G.degree(weight='weight')).values()) / num_nodes if num_nodes > 0 else 0

    results.append({
        'period': period,
        'nodes': num_nodes,
        'edges': num_edges,
        'average_degree': round(avg_degree, 2),
        'average_weighted_degree': round(avg_weighted_degree, 2)
    })

# Simpan ke DataFrame
metrics_df = pd.DataFrame(results)

# Simpan ke file Excel
metrics_df.to_excel('/content/network_properties_fobvalue_coalm_07.xlsx', index=False)

# Tampilkan 5 baris pertama
print(metrics_df.head())

FileNotFoundError: [Errno 2] No such file or directory: '/content/cleaned_coalm 07.xlsx'

In [ ]:
import pandas as pd
import networkx as nx
from networkx.algorithms.community import louvain_communities
from networkx.algorithms.community.quality import modularity

# Load file hasil cleaning
file_path = "/content/cleaned_coalm 07.xlsx"
df = pd.read_excel(file_path)

# Normalisasi kolom
df.columns = [col.lower() for col in df.columns]
df['period'] = df['period'].astype(str)

# Kolom penting
period_col = 'period'
source_col = 'reporterdesc'
target_col = 'partnerdesc'
weight_col = 'fobvalue'

# Simpan hasil
results = []

for period in sorted(df[period_col].unique()):
    df_month = df[df[period_col] == period]

    # Buat directed graph berbobot
    G = nx.DiGraph()
    edges = list(zip(df_month[source_col], df_month[target_col], df_month[weight_col]))
    G.add_weighted_edges_from(edges)

    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    avg_degree = sum(dict(G.degree()).values()) / num_nodes if num_nodes > 0 else 0
    avg_weighted_degree = sum(dict(G.degree(weight='weight')).values()) / num_nodes if num_nodes > 0 else 0
    density = nx.density(G)

    # Konversi ke undirected graph untuk modularity
    if num_nodes > 1 and num_edges > 0:
        UG = G.to_undirected()
        communities = louvain_communities(UG, weight='weight', resolution=1)
        mod = modularity(UG, communities, weight='weight')
    else:
        mod = 0

    results.append({
        'period': period,
        'nodes': num_nodes,
        'edges': num_edges,
        'average_degree': round(avg_degree, 2),
        'average_weighted_degree': round(avg_weighted_degree, 2),
        'density': round(density, 4),
        'modularity': round(mod, 4)
    })

# Simpan ke DataFrame
metrics_df = pd.DataFrame(results)

# Simpan ke Excel
output_path = "/content/network_full_properties_coalm_07.xlsx"
metrics_df.to_excel(output_path, index=False)

# Tampilkan 5 baris pertama
print(metrics_df.head())

import pandas as pd

# df = data mentah sebelum cleaning
# filtered_df = data setelah cleaning
# metrics_df = hasil perhitungan jaringan per bulan

with pd.ExcelWriter('/content/network_metrics_summary_coalm_07.xlsx') as writer:
    df.to_excel(writer, sheet_name='Before Cleaning', index=False)
    filtered_df.to_excel(writer, sheet_name='After Cleaning', index=False)
    metrics_df.to_excel(writer, sheet_name='Network Metrics', index=False)

print("✅ Semua data berhasil disimpan ke 'network_metrics_summary_coalm_07.xlsx'")

from google.colab import files
files.download('/content/network_metrics_summary_coalm_07.xlsx')

In [ ]:
import pandas as pd
import networkx as nx

# Load file hasil cleaning
df = pd.read_excel("/content/cleaned_coalm 07.xlsx")
df.columns = [col.lower() for col in df.columns]
df['period'] = df['period'].astype(str)

# Nama kolom utama
period_col = 'period'
source_col = 'reporterdesc'
target_col = 'partnerdesc'
weight_col = 'fobvalue'

# Simpan semua hasil centrality
centrality_results = []

for period in sorted(df[period_col].unique()):
    df_month = df[df[period_col] == period]

    G = nx.DiGraph()
    edges = list(zip(df_month[source_col], df_month[target_col], df_month[weight_col]))
    G.add_weighted_edges_from(edges)

    # Degree basic
    in_deg = dict(G.in_degree())
    out_deg = dict(G.out_degree())
    in_deg_w = dict(G.in_degree(weight='weight'))
    out_deg_w = dict(G.out_degree(weight='weight'))

    # Degree centrality (normalisasi)
    degree_centrality = nx.degree_centrality(G)

    # Betweenness & closeness centrality
    betweenness = nx.betweenness_centrality(G, weight='weight', normalized=True)
    closeness = nx.closeness_centrality(G)

    for node in G.nodes():
        centrality_results.append({
            'period': period,
            'node': node,
            'in_degree': in_deg.get(node, 0),
            'out_degree': out_deg.get(node, 0),
            'degree': in_deg.get(node, 0) + out_deg.get(node, 0),
            'in_degree_weighted': in_deg_w.get(node, 0),
            'out_degree_weighted': out_deg_w.get(node, 0),
            'degree_centrality': degree_centrality.get(node, 0),
            'betweenness_centrality': betweenness.get(node, 0),
            'closeness_centrality': closeness.get(node, 0)
        })

# Simpan ke DataFrame
centrality_df = pd.DataFrame(centrality_results)

# Simpan ke file Excel
centrality_df.to_excel('/content/network_centrality_full_coalm_07.xlsx', index=False)

# Download
from google.colab import files
files.download('/content/network_centrality_full_coalm_07.xlsx')


FileNotFoundError: [Errno 2] No such file or directory: '/content/cleaned_coalm 07.xlsx'